In [ ]:
import os

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

# import Modules.architectures as archit
import Modules.model as model
import Utils.graphTools as graphTools
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import dgl as dgl
import pickle
import datetime
from scipy.io import savemat

#\\\ Separate functions:
from Utils.miscTools import writeVarValues
from Utils.miscTools import saveSeed
import Utils.graphML as gml

from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score, accuracy_score

numberOfTimeStep = 14
carpetas = ["s1", "s2", "s3"]
norm = "normPower2"

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [ ]:
def exp_kernel(train, sigma):

    matrix_train = np.exp(-(train**2)/(2*(sigma**2)))

    x = pd.DataFrame(matrix_train)
    x = np.round(x, 6)

    eigenvalues = np.linalg.eig(x)
    matriz = np.array(x)
    if not (np.sum(np.abs(eigenvalues[0])>0) == x.shape[0]) and (np.array_equal(matriz, matriz.T)):
        print("==============================")
        print("NO Cumple condición de kernel")
        print("==============================")

    return x

def load_data(c, norm):

    dtw = pd.read_csv("../../E2_Standard-GCN/dtw_matrices/"+carpetas[c]+"/tr_AMR_"+norm+".csv")
    dtw = dtw.fillna(0)
    K = exp_kernel(dtw, 1.5)
    K = K-np.eye(K.shape[0])
    threshold_val = 0.975
    
    A = K.copy()
    A[np.abs(A) < threshold_val] = 0

    # Load raw data
    X_train = np.load("../../DATA/" + carpetas[c] + "/X_train_tensor_"+norm+".npy")
    X_train[X_train == 666] = 0
    print("X_train:", X_train.shape)
    X_val = np.load("../../DATA/" + carpetas[c] + "/X_val_tensor_"+norm+".npy")
    X_val[X_val == 666] = 0
    print("X_val:", X_val.shape)
    X_test = np.load("../../DATA/" + carpetas[c] + "/X_test_tensor_"+norm+".npy")
    X_test[X_test == 666] = 0
    print("X_test:", X_test.shape)

    y_train = pd.read_csv("../../DATA/" + carpetas[c] + "/y_train_"+norm+".csv")[['individualMRGerm_stac']]
    y_train = y_train.iloc[0:y_train.shape[0]:numberOfTimeStep].reset_index(drop=True)
    y_train = torch.tensor(y_train['individualMRGerm_stac'], dtype=torch.float32)
    print("y_train:", y_train.shape)
    y_val = pd.read_csv("../../DATA/" + carpetas[c] + "/y_val_tensor_"+norm+".csv")[['individualMRGerm_stac']]
    y_val = torch.tensor(y_val['individualMRGerm_stac'], dtype=torch.float32)
    print("y_val:", y_val.shape)
    y_test = pd.read_csv("../../DATA/" + carpetas[c] + "/y_test_"+norm+".csv")[['individualMRGerm_stac']]
    y_test = y_test.iloc[0:y_test.shape[0]:numberOfTimeStep].reset_index(drop=True)
    y_test = torch.tensor(y_test['individualMRGerm_stac'], dtype=torch.float32)
    
    print("y_test:", y_test.shape)

    # Vectorize each of the train/val/test sets
    n, dim1, dim2 = X_train.shape
    X_train_vec = torch.tensor(X_train.reshape((n, dim1 * dim2)), dtype=torch.float32)
#     X_train_vec = X_train_vec.unsqueeze(2) 
    print("X_train_vec:", X_train_vec.shape)
    
    n, dim1, dim2 = X_val.shape
    X_val_vec = torch.tensor(X_val.reshape((n, dim1 * dim2)), dtype=torch.float32)
#     X_val_vec = X_val_vec.unsqueeze(2) 
    print("X_val_vec:", X_val_vec.shape)
    
    n, dim1, dim2 = X_test.shape
    X_test_vec = torch.tensor(X_test.reshape((n, dim1 * dim2)), dtype=torch.float32)
#     X_test_vec = X_test_vec.unsqueeze(2) 
    print("X_test_vec:", X_test_vec.shape)
    
    return A, X_train_vec, X_val_vec, X_test_vec, y_train, y_val, y_test


def evaluate(predicted_probabilities, target_binary_class, threshold=0.5):
    """
    Compute accuracy between predicted probabilities and target binary class.
    
    Args:
    - predicted_probabilities: A list or NumPy array of predicted probabilities for the positive class.
    - target_binary_class: A list or NumPy array of the actual binary target class (0 or 1).
    - threshold: Threshold for converting probabilities to binary predictions (default is 0.5).
    
    Returns:
    - Accuracy: Proportion of correct predictions.
    """
    # Convert probabilities to binary predictions based on the threshold
    binary_predictions = [1 if p >= threshold else 0 for p in predicted_probabilities]
    
    # Compare binary predictions with the actual target binary class
    correct_predictions = sum(1 for pred, target in zip(binary_predictions, target_binary_class) if pred == target)
    
    # Calculate accuracy
    accuracy = correct_predictions / len(target_binary_class)
    
    return accuracy

In [ ]:
class GatedGCRNNforClassification(nn.Module):
    """
    GatedGCRNNfforClassification: implements the full GCRNN architecture, i.e.
    h_t = sigma(\hat{Q_t}(A(S)*x_t) + \check{Q_t}(B(S)*h_{t-1}))
    y_t = rho(C(S)*h_t)
    where:
     - h_t, x_t, y_t are the state, input and output variables respectively
     - sigma and rho are the state and output nonlinearities
     - \hat{Q_t} and \check{Q_t} are the input and forget gate operators, which could be time, node or edge gates (or
     time+node, time+edge)
     - A(S), B(S) and C(S) are the input-to-state, state-to-state and state-to-output graph filters
     In the classification version of the Gated GCRNN, y_t is a C-dimensional one-hot vector, where C is the number of classes

    Initialization:

        GatedGCRNNforClassification(inFeatures, stateFeatures, inputFilterTaps,
             stateFilterTaps, stateNonlinearity,
             outputNonlinearity,
             dimLayersMLP,
             GSO,
             bias,
             time_gating=True,
             spatial_gating=None,
             mlpType = 'oneMlp'
             finalNonlinearity = None,
             dimNodeSignals=None, nFilterTaps=None,
             nSelectedNodes=None, poolingFunction=None, poolingSize=None, maxN = None)

        Input:
            inFeatures (int): dimension of the input signal at each node
            stateFeatures (int): dimension of the hidden state at each node
            inputFilterTaps (int): number of input filter taps
            stateFilterTaps (int): number of state filter taps 
            stateNonlinearity (torch.nn): sigma, state nonlinearity in the GRNN cell
            outputNonlinearity (torch.nn): rho, module from torch.nn nonlinear activations
            dimLayersMLP (list of int): number of hidden units of the MLP layers
            GSO (np.array): graph shift operator
            bias (bool): include bias after graph filter on every layer
            time_gating (bool): time gating flag, default True
            spatial_gating (string): None, 'node' or 'edge'
            mlpType (string): either 'oneMlp' or 'multipMLP'; 'multipMLP' corresponds to local MLPs, one per node
            finalNonlinearity (torch.nn): nonlinearity to apply to y, if any (e.g. softmax for classification)
            dimNodeSignals (list of int): dimension of the signals at each layer of C(S) if it is a GNN
            nFilterTaps (list of int): number of filter taps on each layer of C(S) if it is a GNN
            nSelectedNodes (list of int): number of nodes to keep after pooling
                on each layer of the C(S) if it is a Selection GNN
            poolingFunction (nn.Module in Utils.graphML): summarizing function of C(S) if it is a GNN with pooling
            poolingSize (list of int): size of the neighborhood to compute the
                summary from at each layer if C(S) is a GNN with pooling
            maxN (int): maximum number of neighborhood exchanges if C(S) is an Aggregation GNN (default: None)

        Output:
            nn.Module with a full GRNN architecture, state + output neural networks,
            with the above specified characteristics.

    Forward call:

        GatedGCRNNforClassification(x)

        Input:
            x (torch.tensor): input data of shape
                batchSize x seqLength x dimFeatures x numberNodes

        Output:
            y (torch.tensor): output data of shape
                batchSize x seqLength x dimFeatures x numberNodes
    """

    def __init__(self,
                 # State GCRNN
                 inFeatures, stateFeatures, inputFilterTaps,
                 stateFilterTaps, stateNonlinearity,
                 # Output NN nonlinearity
                 outputNonlinearity,
                 # MLP in the end
                 dimLayersMLP,
                 # Structure
                 GSO,
                 # Bias
                 bias,
                 # Gating
                 time_gating = True,
                 spatial_gating = None,
                 # Final nonlinearity, if any, to apply to y
                 finalNonlinearity = None,
                 # Output NN filtering if output NN is GNN
                 dimNodeSignals=None, nFilterTaps=None,
                 # Output NN pooling is output NN is GNN with pooling
                 nSelectedNodes=None, poolingFunction=None, poolingSize=None, maxN = None):
        
        # Initialize parent:
        super().__init__()
        # Check whether the GSO has features or not. After that, always handle
        # it as a matrix of dimension E x N x N.
        assert len(GSO.shape) == 2 or len(GSO.shape) == 3
        if len(GSO.shape) == 2:
            assert GSO.shape[0] == GSO.shape[1]
            GSO = GSO.reshape([1, GSO.shape[0], GSO.shape[1]]) # 1 x N x N
        else:
            assert GSO.shape[1] == GSO.shape[2] # E x N x N

        # Store the values for the state GRNN (using the notation in the paper):
        self.F_i = inFeatures # Input features
        self.K_i = inputFilterTaps # Filter taps of input filter
        self.F_h = stateFeatures # State features
        self.K_h = stateFilterTaps # Filter taps of state filter
        self.E = GSO.shape[0] # Number of edge features
        self.N = GSO.shape[1] # Number of nodes
        self.bias = bias # Boolean
        self.time_gating = time_gating # Boolean
        self.spatial_gating = spatial_gating # None, "node" or "edge"
        self.S = torch.tensor(GSO).clone()
        self.sigma1 = stateNonlinearity	 
        # Declare State GCRNN
        self.stateGCRNN = GGCRNNCell(self.F_i, self.F_h, self.K_i,
                 self.K_h, self.sigma1, self.time_gating, self.spatial_gating,
                 self.E, self.bias)
        self.stateGCRNN.addGSO(self.S)
        # Dimensions of output GNN's  lfully connected layers or of the output MLP
        self.dimLayersMLP = dimLayersMLP
        # Output neural network nonlinearity
        self.sigma2 = outputNonlinearity
        # Selection/Aggregation GNN parameters for the output neural network (default None)
        self.F_o = dimNodeSignals
        self.K_o = nFilterTaps
        self.nSelectedNodes = nSelectedNodes
        self.rho = poolingFunction
        self.alpha = poolingSize
        self.maxN = maxN
        # Nonlinearity to apply to the output, e.g. softmax for classification (default None)
        self.sigma3 = finalNonlinearity    
        
        #\\ If output neural network is MLP:
        if dimNodeSignals is None and nFilterTaps is None:
            fc = []
            if len(self.dimLayersMLP) > 0: # Maybe we don't want to MLP anything
                # The first layer has to connect whatever was left of the graph
                # signal, flattened.
                dimInputMLP = self.N * self.F_h
                # (i.e., N nodes, each one described by F_h features,
                # which means this will be flattened into a vector of size
                    # N x F_h)
                fc.append(nn.Linear(dimInputMLP, self.dimLayersMLP[0], bias = self.bias))
                for l in range(len(dimLayersMLP)-1):
                    # Add the nonlinearity because there's another linear layer
                    # coming
                    fc.append(self.sigma2())
                    # And add the linear layer
                    fc.append(nn.Linear(dimLayersMLP[l], dimLayersMLP[l+1],
                                        bias = self.bias))   
            # And we're done
            # Declare output MLP
            if self.sigma3 is not None :   
                fc.append(self.sigma3())
            self.outputNN = nn.Sequential(*fc)
            # so we finally have the architecture.


    def forward(self, x, h0):
        # Now we compute the forward call
        H = self.stateGCRNN(x,h0)
        h = H.select(1,-1)
        if self.F_o is None: # outputNN is MLP
            h = h.view(-1,self.F_h*self.N)
            y = self.outputNN(h)
        else:
            y = self.outputNN(h)
        return y

    def to(self, device):
        # Because only the filter taps and the weights are registered as
        # parameters, when we do a .to(device) operation it does not move the
        # GSOs. So we need to move them ourselves.
        # Call the parent .to() method (to move the registered parameters)
        super().to(device)
        # Move the GSO
        self.S = self.S.to(device)
        
        
class GGCRNNCell(nn.Module):
    """
    GGCRNNCell Creates a gated recurrent layer that computes h_t = sigma(\hat{Q_t}(A(S)*x_t)
    + \check{Q_t}(B(S)*h_{t-1})), where sigma is a nonlinearity (e.g. tanh), \hat{Q_t} and
    \check{Q_t} are the input and forget gate operators, A(S) and B(S) are LSI-GFs and h_t and x_t are the state and
    input variables respectively. \hat{Q_t} and \check{Q_t} can be time, node or edge gates (or time+node, time+edge).

    Initialization:

        GGCRNNCell(in_features, state_features, in_filter_taps,
                    state_filter_taps, sigma, time_gating=True, spatial_gating=None,
                    edge_features=1, bias=True)

        Inputs:
            in_features (int): number of input features (each feature is a graph
                signal)
            state_features (int): number of state features (each feature is a
                graph signal)
            in_filter_taps (int): number of filter taps of the input filter
            state_filter_taps (int): number of filter taps of the state filter
            sigma (torch.nn): state nonlinearity (default tanh)
            time_gating (bool) = flag for time gating (default True)
            spatial_gating (string) = 'node' or 'edge' gating (default None)
            edge_features (int): number of features over each edge
            bias (bool): add bias vector (one bias per feature) after graph
                filtering

        Output:
            torch.nn.Module for a gated graph recurrent layer.

        Observation: Input filter taps have shape
            state_features x edge_features x filter_taps x in_features

            State filter taps have shape
            state_features x edge_features x filter_taps x state_features

    Add graph shift operator:

        GGCRNNCell.addGSO(GSO) Before applying the filter, we need to define
        the GSO that we are going to use. This allows to change the GSO while
        using the same filtering coefficients (as long as the number of edge
        features is the same; but the number of nodes can change).

        Here we also define the GCRNNs for the input and forget gates, as they
        use the same GSO

        Inputs:
            GSO (torch.tensor): graph shift operator; shape:
                edge_features x number_nodes x number_nodes

    Forward call:

        H = GGCRNNCell(X, h0)

        Inputs:
            X (torch.tensor): input data; shape:
                batch_size x sequence_length x in_features x number_nodes
            h0 (torch.tensor): initial hidden state; shape:
                batch_size x state_features x number_nodes

        Outputs:
            H (torch.tensor): output; shape:
                batch_size x sequence_length x state_features x number_nodes
                
    """

    def __init__(self, G, F, Kin, Kst, sigma=nn.Tanh, time_gating=True, spatial_gating=None, E=1, bias=True):
        # G: number of input features
        # F: number of state features
        # Kin, Kst: number of filter taps
        # sigma: nonlinearity (default tanh)
        # GSOs will be added later.
        # This combines both weight scalars and weight vectors.
        # Bias will always be shared and scalar.

        # Initialize parent
        super().__init__()
        # Save parameters:
        self.G = G
        self.F = F
        self.Kin = Kin
        self.Kst = Kst
        self.E = E
        self.S = None  # No GSO assigned yet
        # Create parameters:
        self.weight_A = nn.parameter.Parameter(torch.Tensor(F, E, Kin, G))
        self.weight_B = nn.parameter.Parameter(torch.Tensor(F, E, Kst, F))
        self.sigma = sigma
        self.time_gating = time_gating
        self.spatial_gating = spatial_gating
        self.bias_flag = bias

        if bias:
            self.bias = nn.parameter.Parameter(torch.Tensor(F, 1))
        else:
            self.register_parameter('bias', None)
        # Initialize parameters
        self.reset_parameters()

    def reset_parameters(self):
        # Taken from _ConvNd initialization of parameters:
        stdv = 1. / math.sqrt(self.G * self.Kin)
        self.weight_A.data.uniform_(-stdv, stdv)
        self.weight_B.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def addGSO(self, S):
        # Every S has 3 dimensions.
        assert len(S.shape) == 3
        # S is of shape E x N x N
        assert S.shape[0] == self.E
        self.N = S.shape[1]
        assert S.shape[2] == self.N
        self.S = S.clone()

    def forward(self, X, h0):
        # X is of shape: batchSize x seqLen x dimInFeatures x numberNodesIn
        # h0 is of shape: batchSize x dimStFeatures x numberNodesIn
        assert h0.shape[0] == X.shape[0]
        B = X.shape[0]
        T = X.shape[1]
        F_in = X.shape[2]

        # Defaults when there is no time gating
        in_gate = torch.ones([B, 1]);
        forget_gate = torch.ones([B, 1]);

        # Compute the GCRNN cell output
        H = torch.empty(0)  # state sequence
        h = h0  # current state
        for i in range(T):
            # Slice input at time t
            x = torch.narrow(X, 1, i, 1)
            x = x.view(B, F_in, self.N)
           
            # Just time gating (if True)
            # Apply filters
            h = in_gate.view(-1, 1, 1).repeat(1, self.F, self.N) * LSIGF(self.weight_A, self.S, x, self.bias) + \
                forget_gate.view(-1, 1, 1).repeat(1, self.F, self.N) * LSIGF(self.weight_B, self.S, h, self.bias)
            # Apply hyperbolic tangent
            h = self.sigma(h)

            h_unsq = h.unsqueeze(1)  # re-create sequence dimension
            # concatenate last state, h, with memory sequence H
            H = torch.cat([H, h_unsq], 1)
        return H
    
    
def LSIGF(h, S, x, b=None):
    """
    LSIGF(filter_taps, GSO, input, bias=None) Computes the output of a linear
        shift-invariant graph filter on input and then adds bias.

    Denote as G the number of input features, F the number of output features,
    E the number of edge features, K the number of filter taps, N the number of
    nodes, S_{e} in R^{N x N} the GSO for edge feature e, x in R^{G x N} the
    input data where x_{g} in R^{N} is the graph signal representing feature
    g, and b in R^{F x N} the bias vector, with b_{f} in R^{N} representing the
    bias for feature f.

    Then, the LSI-GF is computed as
        y_{f} = \sum_{e=1}^{E}
                    \sum_{k=0}^{K-1}
                    \sum_{g=1}^{G}
                        [h_{f,g,e}]_{k} S_{e}^{k} x_{g}
                + b_{f}
    for f = 1, ..., F.

    Inputs:
        filter_taps (torch.tensor): array of filter taps; shape:
            output_features x edge_features x filter_taps x input_features
        GSO (torch.tensor): graph shift operator; shape:
            edge_features x number_nodes x number_nodes
        input (torch.tensor): input signal; shape:
            batch_size x input_features x number_nodes
        bias (torch.tensor): shape: output_features x number_nodes
            if the same bias is to be applied to all nodes, set number_nodes = 1
            so that b_{f} vector becomes b_{f} \mathbf{1}_{N}

    Outputs:
        output: filtered signals; shape:
            batch_size x output_features x number_nodes
    """
    # The basic idea of what follows is to start reshaping the input and the
    # GSO so the filter coefficients go just as a very plain and simple
    # linear operation, so that all the derivatives and stuff on them can be
    # easily computed.

    # h is output_features x edge_weights x filter_taps x input_features
    # S is edge_weighs x number_nodes x number_nodes
    # x is batch_size x input_features x number_nodes
    # b is output_features x number_nodes
    # Output:
    # y is batch_size x output_features x number_nodes

    # Get the parameter numbers:
    F = h.shape[0]
    E = h.shape[1]
    K = h.shape[2]
    G = h.shape[3]
    assert S.shape[0] == E
    N = S.shape[1]
    assert S.shape[2] == N
    B = x.shape[0]
    assert x.shape[1] == G
    assert x.shape[2] == N
    # Or, in the notation we've been using:
    # h in F x E x K x G
    # S in E x N x N
    # x in B x G x N
    # b in F x N
    # y in B x F x N

    # Now, we have x in B x G x N and S in E x N x N, and we want to come up
    # with matrix multiplication that yields z = x * S with shape
    # B x E x K x G x N.
    # For this, we first add the corresponding dimensions
    x = x.reshape([B, 1, G, N])
    S = S.reshape([1, E, N, N])
    z = x.reshape([B, 1, 1, G, N]).repeat(1, E, 1, 1, 1) # This is for k = 0
    # We need to repeat along the E dimension, because for k=0, S_{e} = I for
    # all e, and therefore, the same signal values have to be used along all
    # edge feature dimensions.
    for k in range(1,K):
        x = x.to(S.dtype)
        x = torch.matmul(x, S) # B x E x G x N
        xS = x.reshape([B, E, 1, G, N]) # B x E x 1 x G x N
        z = torch.cat((z, xS), dim = 2) # B x E x k x G x N
    # This output z is of size B x E x K x G x N
    # Now we have the x*S_{e}^{k} product, and we need to multiply with the
    # filter taps.
    # We multiply z on the left, and h on the right, the output is to be
    # B x N x F (the multiplication is not along the N dimension), so we reshape
    # z to be B x N x E x K x G and reshape it to B x N x EKG (remember we
    # always reshape the last dimensions), and then make h be E x K x G x F and
    # reshape it to EKG x F, and then multiply
    z = z.to(h.dtype)
    y = torch.matmul(z.permute(0, 4, 1, 2, 3).reshape([B, N, E*K*G]),
                     h.reshape([F, E*K*G]).permute(1, 0)).permute(0, 2, 1)
    # And permute againt to bring it from B x N x F to B x F x N.
    # Finally, add the bias
    if b is not None:
        y = y + b
    return y

In [ ]:
def createGraph(A):
    # ====> CREATE GRAPH BY ADJACENCY MATRIX
    graphType = 'adjacency'
    graphOptions = {}
    graphOptions['adjacencyMatrix'] = A.values
    G = graphTools.Graph(graphType, A.shape[0], graphOptions)
    G.computeGFT()
    S = G.S/np.abs(np.max(np.diag(G.E)))
    S = np.expand_dims(S,axis=0)
    order = np.arange(G.N)
    ## END CREATE GRAPH <====================
    nNodes = A.shape[0]
    
    return S, nNodes, order


def createModel(input_arguments, params): 
    ################
    # ARCHITECTURE #
    ################

    model = GatedGCRNNforClassification(input_arguments['inFeatures'],
                                     input_arguments['stateFeatures'],
                                     input_arguments['inputFilterTaps'],
                                     input_arguments['stateFilterTaps'],
                                     input_arguments['stateNonlinearity'],
                                     input_arguments['outputNonlinearity'],
                                     input_arguments['dimLayersMLP'],
                                     params['S'],
                                     input_arguments['bias'],
                                     input_arguments['time_gating'],
                                     input_arguments['spatial_gating'])
    # This is necessary to move all the learnable parameters to be
    # stored in the device (mostly, if it's a GPU)
    model.to(params['device'])

    #############
    # OPTIMIZER #
    #############

    if params['trainer'] == 'ADAM':
        optimizer = optim.Adam(model.parameters(),
                                lr = params['learningRate'], weight_decay=params['weight_decay'],
                                betas = (params['beta1'],params['beta2']))
        
    elif params['trainer'] == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=params['learningRate'])
        
    elif params['trainer'] == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(),
                                  lr = params['learningRate'], alpha = params['beta1'])
    

    return model, optimizer


def getbatchIndex(batchSize, nTrain):
    ###########################################
    # DATA INPUT (pick up on data parameters) #
    ###########################################

    # nTrain: size of the training set

    # Number of batches: If the desired number of batches does not split the
    # dataset evenly, we reduce the size of the last batch (the number of
    # samples in the last batch).
    # The variable batchSize is a list of length nBatches (number of batches),
    # where each element of the list is a number indicating the size of the
    # corresponding batch.
    if nTrain < batchSize:
        nBatches = 1
        batchSize = [nTrain]
    elif nTrain % batchSize != 0:
        nBatches = np.ceil(nTrain/batchSize).astype(np.int64)
        batchSize = [batchSize] * nBatches
        # If the sum of all batches so far is not the total number of graphs,
        # start taking away samples from the last batch (remember that we used
        # ceiling, so we are overshooting with the estimated number of batches)
        while sum(batchSize) != nTrain:
            batchSize[-1] -= 1
    # If they fit evenly, then just do so.
    else:
        nBatches = np.int(nTrain/batchSize)
        batchSize = [batchSize] * nBatches
    # batchIndex is used to determine the first and last element of each batch.
    # If batchSize is, for example [20,20,20] meaning that there are three
    # batches of size 20 each, then cumsum will give [20,40,60] which determines
    # the last index of each batch: up to 20, from 20 to 40, and from 40 to 60.
    # We add the 0 at the beginning so that batchIndex[b]:batchIndex[b+1] gives
    # the right samples for batch b.
    batchIndex = np.cumsum(batchSize).tolist()
    batchIndex = [0] + batchIndex
    
    return batchIndex, nBatches, batchSize


def training_Phase(X_train_vec, y_train, X_val_vec, y_val, params, modelName):

    bestMetricDev = float('inf')
    bestHyperparameters = {'lr':-1, 'weight_decay':-1, 'trainer': -1}
    early_stopping_patience = params['early_stopping_patience']
    nTrain = X_train_vec.shape[0]
    nValid = X_val_vec.shape[0]
    weights_to_ADAM = params["weight_decay"]

    for lr in range(len(params["learningRate"])):
        for t in range(len(params["trainer"])):
            if params["trainer"][t] == "ADAM":
                params["weight_decay"] = weights_to_ADAM
            else:
                params["weight_decay"] = [0]
                
            for w in range(len(params["weight_decay"])):
                params_aux = params.copy()
                params_aux["learningRate"] = params["learningRate"][lr]
                params_aux["weight_decay"] = params["weight_decay"][w]
                params_aux["trainer"] = params["trainer"][t]
                print("learningRate: ", params_aux["learningRate"], "- weight_decay:", params_aux["weight_decay"], "- trainer:", params_aux["trainer"])

                model, optimizer = createModel(input_arguments, params_aux)

                batchSize = params['batchSize']
                batchIndex, nBatches, batchSize = getbatchIndex(batchSize, nTrain)

                # Model tracking
                # lossTrain_arr = []
                # accTrain_arr = []
                # lossVal_arr = []
                # accVal_arr = []
                best_validation_loss = float('inf')
                counter_early_stop = 0

                for epoch in range(nEpochs):

                    # Randomize dataset for each epoch
                    randomPermutation = np.random.permutation(nTrain)
                    # Convert a numpy.array of numpy.int into a list of actual int.
                    idxEpoch = [int(i) for i in randomPermutation]

                    # Initialize counter
                    for batch in range(nBatches):
                        # Extract the adequate batch
                        thisBatchIndices = idxEpoch[batchIndex[batch] : batchIndex[batch+1]]
                        # Get the samples
                        xTrain, yTrain = X_train_vec[thisBatchIndices], y_train[thisBatchIndices]
                        xTrain = xTrain.view(batchSize[batch],seqLen,-1)
                        yTrain = yTrain.view(batchSize[batch],-1)

                        # Set the ordering
                        xTrainOrdered = xTrain[:,:,params['order']] # B x F x N

                        # Check if it is an RNN to add sequence dimension
                        if 'RNN' in modelName or 'rnn' in modelName or 'Rnn' in modelName:
                            xTrainOrdered = xTrainOrdered.unsqueeze(2) # To account for just F=1 feature
                            yTrainModel = yTrain.unsqueeze(2) # To account for just F=1 feature
                        else:
                            xTrainOrdered = xTrainOrdered.view(batchSize[batch]*seqLen,1,-1)
                            yTrainModel = yTrain.view(batchSize[batch]*seqLen,1,-1)


                        optimizer.zero_grad()

                        if 'GCRNN' in modelName or 'gcrnn' in modelName or 'GCRnn' in modelName:
                            # Obtain the output of the GCRNN
                            h0 = torch.zeros(batchSize[batch],F1,xTrainOrdered.shape[3])
                            yHatTrain = model(xTrainOrdered, h0)

                        elif 'RNN' in modelName or 'rnn' in modelName or 'Rnn' in modelName:
                            # Obtain the output of the GCRNN
                            h0 = torch.zeros(batchSize[batch],rnnStateFeat)
                            c0 = h0
                            yHatTrain = model(xTrainOrdered, h0, c0)
                        else:
                            # Obtain the output of the GNN
                            yHatTrain = model(xTrainOrdered)
                            yHatTrain = yHatTrain.unsqueeze(1)

                        # Compute loss
                        m = nn.Sigmoid()
                        yHatTrain = m(yHatTrain)
                        if nClass == 2:
                            yHatTrain = yHatTrain[:,1]
                        else:
                            yHatTrain = yHatTrain.view(-1)
                        yTrainModel = yTrainModel.view(-1)
                        lossValueTrain = lossFunction(yHatTrain,yTrainModel)

                        # Compute gradients
                        lossValueTrain.backward()

                        # Optimize
                        optimizer.step()

                        with torch.no_grad():
                            accTrain = evaluate(yHatTrain, yTrainModel)

                        #\\\\\\\
                        #\\\ VALIDATION
                        xValid, yValid = X_val_vec, y_val
                        xValid = xValid.view(nValid,seqLen,-1)
                        yValid = yValid.view(nValid,-1)

                        # Set the ordering
                        xValidOrdered = xValid[:,:,order] # BxFxN
                        xValidOrdered = xValidOrdered.unsqueeze(2)
                        yValidModel = yValid.unsqueeze(2)

                        with torch.no_grad():
                            # Obtain the output of the GNN
                            if 'GCRNN' in modelName or 'gcrnn' in modelName or 'GCRnn' in modelName:
                                h0v = torch.zeros(nValid,F1,xValidOrdered.shape[3])
                                yHatValid = model(xValidOrdered,h0v)
                            elif 'RNN' in modelName or 'rnn' in modelName or 'Rnn' in modelName:
                                # Obtain the output of the GCRNN
                                h0v = torch.zeros(nValid,rnnStateFeat)
                                c0v = h0
                                yHatValid = model(xValidOrdered,h0v,c0v)
                            else:
                                yHatValid = model(xValidOrdered)
                                yHatValid = yHatValid.unsqueeze(1)

                            # Compute loss
                            m = nn.Sigmoid()
                            yHatValid = m(yHatValid)

                            if nClass == 2:
                                yHatValid = yHatValid[:,1]
                            else:
                                yHatValid = yHatValid.view(-1)
                            yValidModel = yValidModel.view(-1)
                            lossValueValid = lossFunction(yHatValid,yValidModel)

                    # Early stopping
                    if lossValueValid < best_validation_loss:
                        best_validation_loss = lossValueValid
                        counter_early_stop = 0
                    else:
                        counter_early_stop += 1

                    if counter_early_stop >= early_stopping_patience:
                        print(f'Early stopping en la epoch {epoch}')
                        break

                    #\\\\\\\
                    #\\\ END OF EPOCH:
                    #\\\\\\\

                if best_validation_loss < bestMetricDev:
                    print("\t\tCambio the best", bestMetricDev, "por metric dev:", best_validation_loss)
                    bestMetricDev = best_validation_loss
                    bestHyperparameters['lr'] = lr
                    bestHyperparameters['weight_decay'] = w
                    bestHyperparameters['trainer'] = t
                
    return bestHyperparameters


def best_Model(X_train_vec, y_train, X_val_vec, y_val, params, bestParams, modelName):

    nTrain = X_train_vec.shape[0]
    nValid = X_val_vec.shape[0]
    finalParams = params.copy()
    finalParams["learningRate"] = params["learningRate"][bestParams["lr"]]
    finalParams["weight_decay"] = params["weight_decay"][bestParams["weight_decay"]]
    finalParams["trainer"] = params["trainer"][bestParams["trainer"]]
    early_stopping_patience = finalParams['early_stopping_patience']

    model, optimizer = createModel(input_arguments, finalParams)

    batchSize = finalParams['batchSize']
    batchIndex, nBatches, batchSize = getbatchIndex(batchSize, nTrain)

    best_validation_loss = float('inf')
    counter_early_stop = 0

    for epoch in range(nEpochs):

        # Randomize dataset for each epoch
        randomPermutation = np.random.permutation(nTrain)
        # Convert a numpy.array of numpy.int into a list of actual int.
        idxEpoch = [int(i) for i in randomPermutation]

        # Initialize counter
        for batch in range(nBatches):
            # Extract the adequate batch
            thisBatchIndices = idxEpoch[batchIndex[batch] : batchIndex[batch+1]]
            # Get the samples
            xTrain, yTrain = X_train_vec[thisBatchIndices], y_train[thisBatchIndices]
            xTrain = xTrain.view(batchSize[batch],seqLen,-1)
            yTrain = yTrain.view(batchSize[batch],-1)

            # Set the ordering
            xTrainOrdered = xTrain[:,:,finalParams['order']] # B x F x N

            # Check if it is an RNN to add sequence dimension
            if 'RNN' in modelName or 'rnn' in modelName or 'Rnn' in modelName:
                xTrainOrdered = xTrainOrdered.unsqueeze(2) # To account for just F=1 feature
                yTrainModel = yTrain.unsqueeze(2) # To account for just F=1 feature
            else:
                xTrainOrdered = xTrainOrdered.view(batchSize[batch]*seqLen,1,-1)
                yTrainModel = yTrain.view(batchSize[batch]*seqLen,1,-1)


            optimizer.zero_grad()

            if 'GCRNN' in modelName or 'gcrnn' in modelName or 'GCRnn' in modelName:
                # Obtain the output of the GCRNN
                h0 = torch.zeros(batchSize[batch],F1,xTrainOrdered.shape[3])
                yHatTrain = model(xTrainOrdered, h0)

            elif 'RNN' in modelName or 'rnn' in modelName or 'Rnn' in modelName:
                # Obtain the output of the GCRNN
                h0 = torch.zeros(batchSize[batch],rnnStateFeat)
                c0 = h0
                yHatTrain = model(xTrainOrdered, h0, c0)
            else:
                # Obtain the output of the GNN
                yHatTrain = model(xTrainOrdered)
                yHatTrain = yHatTrain.unsqueeze(1)

            # Compute loss
            m = nn.Sigmoid()
            yHatTrain = m(yHatTrain)
            if nClass == 2:
                yHatTrain = yHatTrain[:,1]
            else:
                yHatTrain = yHatTrain.view(-1)
            yTrainModel = yTrainModel.view(-1)
            lossValueTrain = lossFunction(yHatTrain,yTrainModel)

            # Compute gradients
            lossValueTrain.backward()

            # Optimize
            optimizer.step()

            #\\\\\\\
            #\\\ VALIDATION
            xValid, yValid = X_val_vec, y_val
            xValid = xValid.view(nValid,seqLen,-1)
            yValid = yValid.view(nValid,-1)

            # Set the ordering
            xValidOrdered = xValid[:,:,order] # BxFxN
            xValidOrdered = xValidOrdered.unsqueeze(2)
            yValidModel = yValid.unsqueeze(2)

            with torch.no_grad():
                # Obtain the output of the GNN
                if 'GCRNN' in modelName or 'gcrnn' in modelName or 'GCRnn' in modelName:
                    h0v = torch.zeros(nValid,F1,xValidOrdered.shape[3])
                    yHatValid = model(xValidOrdered,h0v)
                elif 'RNN' in modelName or 'rnn' in modelName or 'Rnn' in modelName:
                    # Obtain the output of the GCRNN
                    h0v = torch.zeros(nValid,rnnStateFeat)
                    c0v = h0
                    yHatValid = model(xValidOrdered,h0v,c0v)
                else:
                    yHatValid = model(xValidOrdered)
                    yHatValid = yHatValid.unsqueeze(1)

                # Compute loss
                m = nn.Sigmoid()
                yHatValid = m(yHatValid)

                if nClass == 2:
                    yHatValid = yHatValid[:,1]
                else:
                    yHatValid = yHatValid.view(-1)
                yValidModel = yValidModel.view(-1)
                lossValueValid = lossFunction(yHatValid,yValidModel)

        # Early stopping
        if lossValueValid < best_validation_loss:
            best_validation_loss = lossValueValid
            counter_early_stop = 0
        else:
            counter_early_stop += 1

        if counter_early_stop >= early_stopping_patience:
            print(f'Early stopping en la epoch {epoch}')
            break
            
    return model


def test_Phase(model, X_test_vec, y_test, nNodes, results, modelName):
    nTest = X_test_vec.shape[0]
    xTest, yTest = X_test_vec, y_test
    xTest = xTest.view(nTest, seqLen, -1)
    yTest = yTest.view(nTest, -1)

    # Update order and adapt dimensions
    xTestOrdered = xTest[:, :, order]
    if 'RNN' in modelName:
        xTestOrdered = xTestOrdered.unsqueeze(2)
    else:
        xTestOrdered = xTestOrdered.view(nTest, K, -1)

    with torch.no_grad():
        # Process the samples
        if 'GCRNN' in modelName:
            h0t = torch.zeros(nTest, F1, nNodes).to(xTestOrdered.device)  # Ensure device compatibility
            yHatTest = model(xTestOrdered, h0t)
        elif 'RNN' in modelName:
            h0t = torch.zeros(nTest, rnnStateFeat).to(xTestOrdered.device)  # Ensure device compatibility
            c0t = h0t
            yHatTest = model(xTestOrdered, h0t, c0t)
        else:
            yHatTest = model(xTestOrdered)

        m = nn.Sigmoid()
        yHatTest = m(yHatTest)
        yHatTest = yHatTest[:, 1]
        thisAccBest = evaluate(yHatTest, yTest)


    # Convert predictions to binary
    yHatTest = yHatTest.round()

    # Compute ROC AUC score
    roc_auc = roc_auc_score(yTest.detach().numpy(), yHatTest.detach().numpy())

    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(yTest.detach().numpy(), yHatTest.detach().numpy()).ravel()

    # Compute sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    # Compute F1 score
    f1 = f1_score(yTest.detach().numpy(), yHatTest.detach().numpy())

    # Compute accuracy
    accuracy = accuracy_score(yTest.detach().numpy(), yHatTest.detach().numpy())

    # Store results
    results['roc_auc'].append(roc_auc)
    results['sensitivity'].append(sensitivity)
    results['specificity'].append(specificity)
    results['f1_score'].append(f1)
    results['accuracy'].append(accuracy)

    # Print results
    print("ROC AUC:", roc_auc)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

    return results


## Build model: GCRNN + MLP (output)

In [ ]:
#%%##################################################################
#                                                                   #
#                    SETTING PARAMETERS                             #
#                                                                   #
#####################################################################

modelName = "GCRNN"

# Linear layer
seqLen = 14
K = seqLen
F1 = 14 # Number of state features for the GCRNN architectures 
K1 = 2 # Number of filter taps for all filters
rnnStateFeat = 2 # Number of state features for the RNN architecture
nClass = 2 # Number of class

input_arguments = {} 
input_arguments['name'] = 'GCRNN' # Name of the architecture
#\\\ Architecture parameters
input_arguments['inFeatures'] = 1 
input_arguments['stateFeatures'] = F1 
input_arguments['inputFilterTaps'] = K1 
input_arguments['stateFilterTaps'] = K1 
input_arguments['stateNonlinearity'] = torch.tanh
input_arguments['outputNonlinearity'] = nn.ReLU
input_arguments['dimLayersMLP'] = [nClass] 
input_arguments['bias'] = True
input_arguments['time_gating'] = False
input_arguments['spatial_gating'] = None

############
# TRAINING #
############

#\\\ Individual model training options
trainer = ['ADAM','SGD', 'RMSprop'] # Options: 'SGD', 'ADAM', 'RMSprop'
# learningRate = [1e-3, 0.5e-3, 1e-2, 0.5e-2, 1e-1] # In all options
learningRate = [1e-4, 1e-3, 1e-2, 1e-1]
beta1 = 0.9 # beta1 if 'ADAM', alpha if 'RMSprop'
beta2 = 0.999 # ADAM option only
weight_decay = [1e-7, 1e-5, 1e-3, 1e-1, 0]
# weight_decay = [0, 1e-8, 1e-5, 1e-3]

#\\\ Loss function choice
lossFunction = nn.BCELoss()

#\\\ Overall training options
nEpochs = 20 # Number of epochs


params_model = {'learningRate': learningRate, 
          'beta1': beta1, 'beta2': beta2,
          'weight_decay': weight_decay,
          # Others params
          'device':device, 'trainer': trainer, 
          'batchSize': 16, 'early_stopping_patience': 50}

In [ ]:
results = {'roc_auc':[], 'sensitivity':[], 'specificity':[], 'f1_score':[], 'accuracy':[]}
bestHyperparameters = {}

for c in range(len(carpetas)):
    A, X_train_vec, X_val_vec, X_test_vec, y_train, y_val, y_test = load_data(c, norm)
    S, nNodes, order = createGraph(A)
    
    params_model['S'] = S
    params_model['order'] = order

    bestParamsToCreateModel = training_Phase(X_train_vec, y_train, X_val_vec, y_val, params_model, modelName)
    
    params_bestModel = {'learningRate': learningRate, 
          'beta1': beta1, 'beta2': beta2,
          'weight_decay': weight_decay,
          # Others params
          'device':device, 'trainer': trainer, 
          'batchSize': 16, 'early_stopping_patience': 50,
          'S': S, 'order': order}
    
    model = best_Model(X_train_vec, y_train, X_val_vec, y_val, params_bestModel, bestParamsToCreateModel, modelName)
    results = test_Phase(model, X_test_vec, y_test, nNodes, results, modelName)
    bestHyperparameters[carpetas[c]] = bestParamsToCreateModel

In [ ]:
keys = list(results.keys())
for c in range(len(carpetas)):
    print("================= SPLIT " + str(carpetas[c]) + " ===================")
    print(keys[0] + ": " + str(np.round(results[keys[0]][c]*100,3)))
    print(keys[1] + ": " + str(np.round(results[keys[1]][c]*100,3)))
    print(keys[2] + ": " + str(np.round(results[keys[2]][c]*100,3)))
    
print()
keys = list(results.keys())
for i in range(len(results.keys())):
    average = np.mean(results[keys[i]])
    std = np.std(results[keys[i]])
    print(keys[i] + ": " + str(np.round(average*100,2)) + " +- " + str(np.round(std*100, 2)))

In [ ]:
import json
def saveBestHyperparameters(best_hyperparameters, nombre):
    with open(nombre, 'w') as archivo:
        json.dump(best_hyperparameters, archivo, indent=4)
        
def loadBestHyperparameters(nombre):
    with open(nombre, 'r') as archivo:
        hyperparameteres = json.load(archivo)
        
    return hyperparameteres


In [ ]:
saveBestHyperparameters(bestHyperparameters, "GRNN_bestHyperparameters")
saveBestHyperparameters(results, "GRNN_bestResults")